# Partrac Q1 report

In [1]:
# load my library (only functions for now) for processing the data
# Import standard Python libraries
import os
import sys
import glob

# Import opensoundscape modules for audio processing
from opensoundscape import Audio, audio, Spectrogram
import opensoundscape

# Import utility libraries
import warnings

# Import scientific computing libraries
import numpy as np
import scipy.signal as signal
import importlib.util
import matplotlib.pyplot as plt
import math

warnings.filterwarnings("ignore")
# Enable memory profiling
%load_ext memory_profiler

C:\Users\DrYangYang\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\DrYangYang\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\DrYangYang\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# Add pamseek directory to Python's module search path
pamseek_path = r"C:\Users\DrYangYang\Documents\Python\pamseek"
if pamseek_path not in sys.path:
    sys.path.append(pamseek_path)

from pamseek.utils import load_audio_files, cal_hydrophone
from pamseek.compute import compute_psd, compute_spectrogram
from pamseek.plot import plot_psd, plot_spectrogram, plot_psd_with_percentiles

# 1. Load data 

In [4]:
%%time
%%memit

path = r'C:\Users\DrYangYang\Documents\Python\JupyterNotebook\data'
# Then use it as:
audio_object = load_audio_files(path)
audio_object

Reading audio files from: C:\Users\DrYangYang\Documents\Python\JupyterNotebook\data
Found 10 .wav files:
- 20240717T093000_11805655683051522_2.0dB_3.9V_ver3.20.wav
- 20240717T094000_11805655683051522_2.0dB_3.9V_ver3.20.wav
- 20240717T095000_11805655683051522_2.0dB_3.9V_ver3.20.wav
- 20240717T100000_11805655683051522_2.0dB_3.9V_ver3.20.wav
- 20240717T101000_11805655683051522_2.0dB_3.9V_ver3.20.wav
Concatenation finished --------------- 11%
Concatenation finished --------------- 22%
Concatenation finished --------------- 33%
Concatenation finished --------------- 44%
Concatenation finished --------------- 56%
Concatenation finished --------------- 67%
Concatenation finished --------------- 78%
Concatenation finished --------------- 89%
Concatenation finished --------------- 100%
peak memory: 8694.63 MiB, increment: 8144.39 MiB
CPU times: total: 20.7 s
Wall time: 35 s


In [5]:
# check metadata 
print(f"Entire audio file starts at {audio_object.metadata['recording_start_time']} and has duration {audio_object.duration} seconds.")
print(f"Number of samples: {len(audio_object.samples)}")
print(f"Sample rate: {audio_object.sample_rate} Hz")
print(f"RMS: {audio_object.rms:0.3f}")
print(f"dBFS: {audio_object.dBFS:0.1f}") # decibels relative to full scale

Entire audio file starts at 2024-07-17 09:30:00+00:00 and has duration 6000.64 seconds.
Number of samples: 576061440
Sample rate: 96000 Hz
RMS: 0.031
dBFS: -27.2


# 2. Calibration 

In [7]:
audio_object = cal_hydrophone(audio_object, -156, gain=0, bit_depth=24)

In [8]:
print(f"RMS: {audio_object.rms:0.3f}")
print(f"dBFS: {audio_object.dBFS:0.1f}")

RMS: 0.233
dBFS: -9.6


# 3. Compute PSD

In [ ]:
%%time
%%memit

f1, Pxx = compute_psd(audio_object, 
                           fs=None, 
                           window='hann', 
                           window_length=1.0,
                           overlap=0.5, 
                           scaling='density')

f2, t, Sxx, rms, percentile = compute_spectrogram(audio_object, 
                           fs=None, 
                           window='hann', 
                           window_length=1.0,
                           overlap=0.5, 
                           scaling='density')

In [19]:
Sxx = compute_spectrogram(audio_object, 
                           fs=None, 
                           window='hann', 
                           window_length=1.0,
                           overlap=0.5, 
                           scaling='density')

In [29]:
Sxx

(array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 4.7998e+04, 4.7999e+04,
        4.8000e+04]),
 array([5.0000e-01, 1.0000e+00, 1.5000e+00, ..., 5.9990e+03, 5.9995e+03,
        6.0000e+03]),
 array([[ 25.877716 ,  18.184454 ,  37.379997 , ...,  27.62964  ,
          32.59671  ,  11.775984 ],
        [ 42.57425  ,  40.651985 ,  41.75572  , ...,  33.508293 ,
          42.629547 ,  43.822056 ],
        [ 42.231792 ,  47.345314 ,  40.522095 , ...,  41.827488 ,
          36.903557 ,  47.58316  ],
        ...,
        [  9.821914 ,  10.53986  ,   2.2608669, ...,   6.1759753,
           8.636244 ,   3.567565 ],
        [ 12.650199 ,  -2.718651 ,  13.584295 , ...,   2.4592357,
           1.2105881,   0.5494997],
        [ 10.400009 ,  11.721205 ,   3.3179429, ..., -28.750708 ,
          -9.694365 , -10.386532 ]], dtype=float32),
 array([83.47972 , 89.87039 , 93.8006  , ..., 23.347063, 23.510357,
        21.014538], dtype=float32),
 {'1%': array([  4.23277191,  26.52928104,  27.87089832, ..., -1

# 4. Compute 1/3 band, boardband 

# 5. Plot

In [34]:
from pamseek.plot import plot_psd, plot_spectrogram, plot_psd_with_percentiles

(array([0.0000e+00, 1.0000e+00, 2.0000e+00, ..., 4.7998e+04, 4.7999e+04,
        4.8000e+04]),
 array([101.005905, 107.39659 , 111.3268  , ...,  40.873264,  41.03655 ,
         38.540745], dtype=float32))

In [ ]:
%%time
%%memit

plot_psd(f1, Pxx, xscale='log', yscale='linear', width=12, height=6,
             title='Power Spectral Density', grid=True, xlim=None, ylim=None, 
             save=False, filename='psd_plot.png', dpi=300, color='b')

In [ ]:
%%time
%%memit

plot_spectrogram(f1, t, Sxx, xscale='linear', yscale='log', width=12, height=6,
                    title='Spectrogram', grid=True, xlim=None, ylim=[1, 12000], 
                    cmap='viridis', vmin=None, vmax=None, colorbar_label='PSD [dB re 1 µPa²/Hz]',
                    save=False, filename='spectrogram_plot.png', dpi=300)